In [ ]:
# Installation for IOC defanging - https://github.com/ioc-fang/ioc-fanger
!pip install -q ioc-fanger

In [ ]:
import datetime
import pandas as pd
import ioc_fanger
import json
import time
import re
import os
import zipfile
from collections import defaultdict
from nltk.tokenize import word_tokenize
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
import concurrent.futures
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

## Generate complete list of HF keywords

In [ ]:
tags = ["MAL", "FILE", "OS", "PROT", "MD5", "SHA1", "SHA2",  "ENCR", "TOOL", "VULID", "VULNAME",
        "ACT", "APT", "SECTEAM", "IDTY", "EMAIL", "IP", "DOM", "URL",  ]  #"LOC", "TIME", 

chunk_size = 500

def save_chunk(output_directory, count, chunk):
    output_file = os.path.join(output_directory, f"chunk_{count}.txt")
    with open(output_file, 'w', encoding='utf-8') as file:
        file.write('\n'.join(chunk))

output_directory = 'new_lists_folder'
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

output_directory = 'chunks_complete_entity_list'
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

complete_list = set()
idx_file = 1
count = 0
chunk = []
tag_list = []
for t in tags:
    tag = t
    print(f'Tag {t} first file index {idx_file}')
    countl=0
    new_list = []
    file_path = f'/home/anon/input/datasets-extracted-entities/REP_extracted_entities/entity_lists/{t}_list.txt'
    with open(file_path, 'r', encoding='utf-8') as file:
        new_file = []
        for line in file:
            new_file.append(line.strip())
        new_file = sorted(new_file, key=lambda x: (-len(x), x))
        for line in new_file:
            countl += 1
            line = line.strip()
            if line not in complete_list:
                complete_list.add(line)
                new_list.append(line)
                
                chunk.append(line)
                count += 1
                if count % chunk_size == 0:
                    save_chunk('chunks_complete_entity_list', idx_file, chunk)
                    tag_list.append(tag)
                    chunk = []
                    idx_file += 1                    
        if chunk:
            save_chunk('chunks_complete_entity_list', idx_file, chunk)
            tag_list.append(tag)
            chunk = []
            idx_file += 1
            count=0
    
    print(f'{t} - Before: {countl} - After: {len(new_list)}\n')
    
    output_file = os.path.join('new_lists_folder', f"{t}_list.txt")
    with open(output_file, 'w', encoding='utf-8') as file:
        file.write('\n'.join(new_list))

print(f'Chunks: {idx_file}')

In [ ]:
len(tag_list)

## Search REP keywords in REP articles

In [ ]:
input_directory_keywords = '/home/anon/working/chunks_complete_entity_list'
input_path_REP = '/home/anon/input/datasets-extracted-entities/REP_extracted_entities/REP_extracted_entities.json'
output_directory = '/home/anon/working/chunks_REP_keywords_in_REP_articles/'
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

### Load REP dataframe

In [ ]:
# Load HF posts
df_REP = pd.read_json(input_path_REP)

print('Number of relevant articles: ', df_REP.shape[0])

In [ ]:
def defang_iocs_in_text(text):
    defanged_text = ioc_fanger.defang(text)
    return defanged_text

def fang_iocs_in_text(text):
    fanged_text = ioc_fanger.fang(text)
    return fanged_text

In [ ]:
# Function to merge sentence lists
def merge_sentence_list(article):
    merged_list = ' '.join(article).lower()
    return merged_list

# Function to merge entity lists
def merge_entity_lists(entities):
    merged_list = []
    for entity_type, entity_list in entities.items():
        merged_list.extend([fang_iocs_in_text(el).lower() for el in entity_list])
    return merged_list


#df_REP['merged_content'] = df_REP['flatContent'].apply(merge_sentence_list)
df_REP['merged_content'] = df_REP['content'].apply(merge_sentence_list)


df_REP['date'] = pd.to_datetime(df_REP['date'])
df_REP['date'] = df_REP['date'].dt.strftime('%m-%d-%Y')

# Sort the DataFrame by the 'date' column
df_REP.sort_values(by='date', inplace=True)

# Apply the merge_entity_lists function to each row and create a new column
df_REP['merged_entities'] = df_REP['entities'].apply(merge_entity_lists)

# Filter the DataFrame to only keep the selected columns
df_REP = df_REP[['ID', 'date', 'merged_entities', 'merged_content']]

# Convert the filtered DataFrame to a list of dictionaries
existing_json_data = df_REP.to_dict(orient='records')

### Search with entities

In [ ]:
def process_keyword(keyword):
    keyword_list = []
    for entry in existing_json_data:
        if keyword in entry['merged_entities']:
            keyword_list.append({
                'ID': entry['ID'],
                'date': entry['date']
            })
    return keyword, keyword_list

In [ ]:
# Define regular expression patterns
ip_pattern = re.compile(r'^(\d+\.\d+\.\d+\.\d+)$')
pattern_to_remove = re.compile(r'^[-\'\+,/\\:=_\|\.\^~0-9]+$')
stop_words = set(stopwords.words('english'))

In [ ]:
def get_REP_keywords_in_REP(keywords_path):
    print(f'Working on {keywords_path}')
    # Open the set of keywords
    with open(keywords_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    # Remove newlines and create a list
    keywords_list = [line.strip() for line in lines]
    # Filter and keep only the words that match the patterns
    keywords_list = [keyword for keyword in keywords_list
                     if ip_pattern.match(keyword) or not (pattern_to_remove.match(keyword) or keyword in stop_words or len(keyword) == 1)]

    # Create a dictionary to hold the reports by keyword
    return {keyword: [] for keyword in keywords_list}, keywords_list

In [ ]:
def start_searching(REP_keywords_in_REP_dict, tag, keywords_list):
    # Use ThreadPoolExecutor to parallelize keyword processing
    with concurrent.futures.ThreadPoolExecutor() as executor:
        # Submit tasks for each keyword
        keyword_futures = {executor.submit(process_keyword, keyword): keyword for keyword in keywords_list}

        # Use tqdm to track progress
        with tqdm(total=len(keywords_list), desc='Searching in entities') as pbar:
            for future in concurrent.futures.as_completed(keyword_futures):
                keyword = keyword_futures[future]
                keyword_results = future.result()
                REP_keywords_in_REP_dict[keyword] = keyword_results
                pbar.update(1)  # Update progress bar
    
    # Save resulting list
    final_structure = []

    for keyword, value_list in REP_keywords_in_REP_dict.items():
            final_structure.append({
                'keyword': keyword,
                'tag': tag,
                'matching_posts': value_list[1]
            })
            
    return final_structure

### Search with articles

In [ ]:
def read_lines_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            yield line.strip()

def create_set_from_file(file_path):
    return set(read_lines_from_file(file_path))

def create_bulk_search_pattern(keywords_list):
    keywords_list = sorted(keywords_list, key=lambda x: (-len(x), x))
    #keyword_regex = r"\b(?:" + "|".join(re.escape(word) for word in keywords_list) + r")\b"
    keyword_regex = r"(^|\s)[^a-zA-Z0-9]*(" + "|".join(re.escape(word.lower()) for word in keywords_list) + r")[^a-zA-Z0-9]*(\s|$)"
    return re.compile(keyword_regex)

def search_matches_in_chunk(chunk, bulk_search_pattern):
    word_matching = defaultdict(list)
    for post in chunk.itertuples(index=False):
        content = post.merged_content.lower()

        matches = bulk_search_pattern.findall(content)
        for word in matches:
            matched_word = word[1].lower()
            word_matching[matched_word].append({
                'ID': post.ID,
                'date': post.date
            })
    return word_matching

def search_matches(input_file_path, output_file_path, df_REP):
    # Load keywords of reports
    keywords_list = create_set_from_file(input_file_path)

    # Convert keywords_list to lowercase for efficient matching
    keywords_list_lower = {word.lower() for word in keywords_list}

    # Compile the bulk search pattern
    bulk_search_pattern = create_bulk_search_pattern(keywords_list_lower)

    # Divide the dataframe into smaller chunks for parallel processing
    chunk_size = 1000
    df_chunks = [df_REP[i:i+chunk_size] for i in range(0, df_REP.shape[0], chunk_size)]
                
    # Use ThreadPoolExecutor for parallel processing
    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(search_matches_in_chunk, chunk, bulk_search_pattern) for chunk in df_chunks]
        word_matching_dict = {}
        for future in tqdm(futures, total=len(futures), desc='Searching in articles'):
            chunk_word_matching = future.result()
            for word, matches in chunk_word_matching.items():
                if word in word_matching_dict:
                    word_matching_dict[word].extend(matches)
                else:
                    word_matching_dict[word] = matches

    # Convert word_matching_dict to a list of dictionaries
    word_matching = [{'keyword': word, 'matching posts': matches} for word, matches in word_matching_dict.items()]

    return word_matching

### Compare

In [ ]:
def compare_struct_and_save(struct_from_entity, struct_from_posts, output_path):
    for entry in struct_from_posts:
        kw = entry['keyword']
        mp = entry['matching posts']
        
        for idx, item in enumerate(struct_from_entity):
            if item['keyword'] == kw:
                item['matching_posts'].extend(mp)
                break
                
    
    # Create a dictionary to store unique IDs for each keyword using sets
    unique_ids = {}
    # Iterate through the data and remove duplicates using sets
    for item in tqdm(struct_from_entity, 'Removing duplicates'):
        keyword = item["keyword"]
        #tag = item["tag"]
        matching_posts = item["matching_posts"]
        # Create a set to store unique IDs for the current keyword
        unique_ids[keyword] = set()
        # Create a list to store unique posts for the current keyword
        unique_posts = []
        for post in matching_posts:
            post_id = post["ID"]
            # Check if the ID is not in the unique_ids set for this keyword
            if post_id not in unique_ids[keyword]:
                # Add the ID to the unique_ids set
                unique_ids[keyword].add(post_id)
                # Add the post to the unique_posts list
                unique_posts.append(post)
        # Update the matching posts for the current keyword
        item["matching_posts"] = unique_posts
    
    with open(output_path, 'w', encoding='utf-8') as outfile:
        json.dump(struct_from_entity, outfile, indent=4)

### Execute

In [ ]:
start = 1
end = start + 70  
#(1, 71) - (71, 141) - (141, 211) - (211, 281) - (281, 351) - (351, 421) - (421, 491) - (491, 561) - (561, 631) - (631, 698) 

end = min(end, 698)

print('Searching from {0} to {1}'.format(start, end-1))

for idx_file in range(start, end):  
    iFilename = 'chunk_{0}.txt'.format(idx_file)
    input_path = os.path.join(input_directory_keywords, iFilename)    
    REP_keywords_in_REP_dict, keywords_list = get_REP_keywords_in_REP(input_path)

    oFilename = 'REP_keywords_in_REP_articles_{0}.json'.format(idx_file)
    output_path = os.path.join(output_directory, oFilename)
    tag = tag_list[idx_file-1]

    struct_from_entity = start_searching(REP_keywords_in_REP_dict, tag, keywords_list)
    struct_from_posts = search_matches(input_path, output_path, df_REP)
    
    compare_struct_and_save(struct_from_entity, struct_from_posts, output_path)

    print()

## Save as ZIP

In [ ]:
# Directory path containing the output files to zip
source_directory = output_directory

# Zip file path
zip_file_path = '/home/anon/working/HF_keywords_in_HF_posts.zip'

# Create a ZIP archive
with zipfile.ZipFile(zip_file_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    # Loop through all files in the directory and add them to the ZIP archive
    for root, _, files in os.walk(source_directory):
        for file in files:
            file_path = os.path.join(root, file)
            zipf.write(file_path, os.path.relpath(file_path, source_directory))